<a href="https://colab.research.google.com/github/Uzair-DeVops/Quater-2/blob/main/Project_02_LangChain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install langchain_community pypdf  langchain_google_genai  langchain_chroma

In [27]:
!pip install  --upgrade google-auth google-auth-oauthlib google-auth-httplib2  google-cloud


In [28]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/sample_data/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = loader.load()
data

[Document(metadata={'source': '/content/sample_data/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two mac

In [29]:
len(data)

11

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("total number of documents : " ,len(docs))

total number of documents :  43


In [31]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY_3')

In [32]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/sample_data/test.json"


In [33]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [34]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GOOGLE_API_KEY)

In [35]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


In [36]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp" , temperature = 0.3 , max_tokens=1000)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate




system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [38]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
response = rag_chain.invoke({"input" : "what is uzair"})
print(response["answer"])

I'm sorry, but the provided context does not contain any information about "uzair". Therefore, I cannot answer your question.



In [40]:
response = rag_chain.invoke({"input" : "what is multi attention"})
print(response["answer"])

Multi-head attention involves linearly projecting queries, keys, and values multiple times with different learned linear projections. The attention function is then performed in parallel on each of these projected versions, yielding output values that are concatenated and projected again. This allows the model to attend to information from different representation subspaces at different positions.

